# Practica 1B
##  Arturo Ibáñez 
## Jorge del Valle

In [1]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [2]:
class Problem(object):

    def __init__(self, initial, goal=None):
        
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        raise NotImplementedError

    def result(self, state, action):
        raise NotImplementedError

    def goal_test(self, state):
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal
        
    def path_cost(self, c, state1, action, state2):
        return c + 1

    def value(self, state):
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        return 1

## Problema Puente
### Comentarios
#### - Utilizamos tanto bfts y dfts.
#### - No podemos usar graph search puesto que no se puede hacer hash de lista. Igual con a*
#### - Para representar el problema cada estado almacena la gente que queda por cruzar , en que lado del puente se encuentra la linterna y el tiempo de uso de la literna acumulado. El estado objetivo es aquel en el que han cruzado todos la linterna queda a la derecha y les ha durado la linterna encendida hasta el final o sobra algo de batería
#### - La lista cruzar almacena los tiempos que tarda cada persona en cruzar
#### - Las acciones vienen dadas por la persona o la pareja que procede a cruzar el puente
#### - Coste:
#### - Heuristica :

In [11]:
# Creamos la clase ProblemaPuente con los elementos que representarán el problema. 
class Puente(Problem):
    def __init__(self, initial, cruzar, goal=(list(),1,300)):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        self.cruzar= cruzar
        self._actions = ['1','2', '3','4','5','12','23','34','45','13','24','35','14','25','15']

    def goal_test(self, state):
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state[0] == self.goal[0] and state[1] == self.goal[1] and state[2] <= self.goal[2]
    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        return [a for a in self._actions if self._is_valid(s,a)]

    def _is_valid(self, s,a):
        '''Determina si un estado es valido o no.'''
        t1 = self.cruzar[int(a[0])-1]
        t2= 0
        if len(a)>1:
            t2 = self.cruzar[int(a[1])-1]
        t3 = max(t1, t2)
        return (s[2]+ t3 <=self.goal[2]) and ((s[1] == 0 and (int(a[0]) in s[0]) and ((len(a)<2) or (int(a[1]) in s[0]))) or (s[1] == 1 and (not(int(a[0]) in s[0])) and (len(a)<2 or not(int(a[1]) in s[0])))) and  1 <= int(a[0]) <= 5 and (len(a)<2 or 0 <= int(a[1])<= 5) 

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        estado = s
        conjunto=set(estado[0])
        linterna=estado[1]
        tiempo=estado[2]
        t1 = self.cruzar[int(a[0])-1]
        t2= 0
        if len(a)>1:
            t2 = self.cruzar[int(a[1])-1]
        t3 = max(t1, t2)
        if linterna == 0:
            conjunto.discard(int(a[0]))
            if len(a)>1:
                conjunto.discard(int(a[1]))
            return (list(conjunto), 1, tiempo+t3)
        else:
            conjunto.add(int(a[0]))
            if len(a)>1:
                conjunto.add(int(a[1]))
            return (list(conjunto), 0,tiempo+t3)
        
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1

In [4]:
p = Puente(([1,2,3,4,5], 0, 0), [10,30,60,80,120],(list(), 1, 300))

In [5]:
depth_first_tree_search(p).solution()

['13', '1', '12', '2', '45', '1', '12']

In [6]:
breadth_first_tree_search(p).solution()

['12', '1', '45', '2', '12', '1', '13']

## Problema Puzzle matemático
### Comentarios
#### -  Usamos para resolverlo dfs (tree o graph) o bfs (tree)
#### -  Tanto a* como bfgs no llegan a acabar
#### -  Para representarlo hemos considerado 14 casillas, donde una es el resultado, 13 de operandos de los que 4 vienen precolocados
#### - En el estado se guardan las cifras todavía no colocadas, la casilla actual pra conocer la operación cerrespondiente y el valor acumulado. El estado objetivo es aquel donde no quedan  cifras por colocar, se han ocupado todas las casillas y se tiene el valor esperado
#### - El array operaciones concreta cómo es el tablero, es decir, la sucesión de operaciones arealizar
#### - No consideramos la posibilidad de obtener el resultado con algun casilla sin colocar
#### - Las acciones consideradas son o bien operar una casilla precolocadao bien colocar una cifra y realizar la operación correspondiente 
#### - Coste:
#### - Heuristica: 

In [7]:
operaciones = ["Comienzo","Suma 13","Multiplicacion","Division","Suma","Suma 12","Multiplicacion","Resta","Resta 11","Suma","Multiplicacion","Division","Resta 10"]
class Puzzle(Problem):
    """estado : numeros por colocar, casillas con las que hemos operado(incluyendo las prefijadas), valor actual"""
    def __init__ (self,initial = ((1,2,3,4,5,6,7,8,9),0,0),goal = ((),13,66)):
        self.initial= initial
        self.goal = goal
        
    def actions (self,estado):
        indice = estado[1]
        if indice == 1:
            "Suma 13"
            return [operaciones[1]]
        elif indice == 5:
            "Suma 12"
            return [operaciones[5]]
        elif indice == 8:
            "Resta 11"
            return [operaciones[8]]
        elif indice == 12:
            "Resta 10"
            return [operaciones[12]]
        else:
            accs = list()
            for i in range(0,len(estado[0])):
                accs.append("Colocar el numero " + str(estado[0][i]))
            return accs
            
    def result (self,estado,accion):
        l = list(estado[0])
        indice = estado[1]
        num = estado[2]
        if accion == "Suma 13":
            return (tuple(l),indice+1,num+13)
        elif accion == "Suma 12":
            return (tuple(l),indice+1,num+12)
        elif accion == "Resta 11":
            return (tuple(l),indice+1,num-11)
        elif accion == "Resta 10":
            return (tuple(l),indice+1,num-10)
        else:
            for i in range(1,10):
                if accion == "Colocar el numero " + str(i):
                        l.remove(i)
                        if operaciones[indice] == "Comienzo":
                            num = i
                        elif operaciones[indice] == "Suma":
                            num += i
                        elif operaciones[indice] == "Multiplicacion":
                            num *= i
                        elif operaciones[indice] == "Division":
                            num /= i
                        elif operaciones[indice] == "Resta":
                            num -= i
            indice += 1
            return (tuple(l),indice,num)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1

In [8]:
breadth_first_tree_search(Puzzle()).solution()

['Colocar el numero 1',
 'Suma 13',
 'Colocar el numero 2',
 'Colocar el numero 4',
 'Colocar el numero 7',
 'Suma 12',
 'Colocar el numero 5',
 'Colocar el numero 8',
 'Resta 11',
 'Colocar el numero 3',
 'Colocar el numero 6',
 'Colocar el numero 9',
 'Resta 10']

In [9]:
depth_first_graph_search(Puzzle()).solution()

['Colocar el numero 9',
 'Suma 13',
 'Colocar el numero 7',
 'Colocar el numero 2',
 'Colocar el numero 8',
 'Suma 12',
 'Colocar el numero 4',
 'Colocar el numero 3',
 'Resta 11',
 'Colocar el numero 6',
 'Colocar el numero 1',
 'Colocar el numero 5',
 'Resta 10']

In [10]:
depth_first_tree_search(Puzzle()).solution()

['Colocar el numero 9',
 'Suma 13',
 'Colocar el numero 7',
 'Colocar el numero 2',
 'Colocar el numero 8',
 'Suma 12',
 'Colocar el numero 4',
 'Colocar el numero 3',
 'Resta 11',
 'Colocar el numero 6',
 'Colocar el numero 1',
 'Colocar el numero 5',
 'Resta 10']